In [1]:
import pandas as pd
import numpy as np
import collections

## 統計各牙齒種類的準確率

In [8]:
molar    = [ 1, 2, 3, 14, 15, 16, 17, 18, 19, 30, 31, 32]
premolar = [ 4, 5, 12, 13, 20, 21, 28, 29 ]
canine   = [ 6, 11, 22, 27  ]
incisors = [ 7, 8, 9, 10, 23, 24, 25, 26 ]


tooth_type_dict = { tooth_type: tooth_list for tooth_type, tooth_list in 
                    zip([ "molar", "premolar", "canine", "incisors" ],\
                        [molar, premolar, canine, incisors])  
                  }

def accuracy(data, title, filter_list):
        total_data = data[ (data["tooth_num"].isin(filter_list)) ]
        match_data = data[ (data["tooth_num"].isin(filter_list)) & (data["Match"] == 1) ]
        
        
        if len(total_data) == 0 : accuracy = 0
        else : accuracy = round(len(match_data) / len(total_data) * 100, 2)
        
        return len(total_data), len(match_data), accuracy
#         print(f"Accuracy for {title} : {accuracy} %")

fold_accuracy_table, data_volume_table, match_table = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
accuracy_dict, data_volume_dict, match_dict = collections.OrderedDict(), collections.OrderedDict(), collections.OrderedDict()

root_dir = "Inception_Three_new_Balance_Common_Class3_Meng_Sheng_Chang_Fold_520201123-135039"
# root_dir = "Inception_Three_new_Balance_Common_Class3_Chang_Majority_Fold_520201124-084205"
# root_dir = "Inception_Three_new_Balance_Common_Multi_Class3_Chang_Majority_Fold_520201124-210453"

for fold_num in range(1, 6):
        path = f"Results/{root_dir}/Fold_{fold_num}/table/test_predict.csv"
        data = pd.read_csv(path)
        match = [ 1 if gt == predict else 0 for gt, predict in zip(data.Class, data.Predict) ]
        data["Match"] = match
        
        item, item_volume, item_match = {}, {}, {}
        for tooth_type, tooth_list in tooth_type_dict.items():
                total, match, acc = accuracy(data, tooth_type, tooth_list)
                item_volume[tooth_type] = total
                item_match[tooth_type]  = match
                item[tooth_type] = acc
        
        item[f"Total_Acc"] = round( len( data[ data["Match"] == 1 ] ) / len( data ) * 100, 2)      
        
        
        accuracy_dict[ f"Fold_{fold_num}" ]     = item
        data_volume_dict[ f"Fold_{fold_num}" ]  = item_volume
        match_dict[ f"Fold_{fold_num}" ]        = item_match
 

results = fold_accuracy_table.from_dict(accuracy_dict).T
accuracy_dict["Tooth_Acc"] = round(results.mean(axis=0), 2)
results = fold_accuracy_table.from_dict(accuracy_dict).T

results

,molar,premolar,canine,incisors,Total_Acc
Fold_1,63.26,68.15,79.15,70.25,67.73
Fold_2,69.41,71.82,66.28,72.98,70.92
Fold_3,63.98,75.25,71.98,74.96,71.11
Fold_4,66.79,71.16,75.88,63.42,67.81
Fold_5,63.80,59.76,58.74,71.45,64.82
Tooth_Acc,65.45,69.23,70.41,70.61,68.48


In [3]:
data_volume_results = data_volume_table.from_dict(data_volume_dict).T
data_volume_results

,molar,premolar,canine,incisors
Fold_1,8617,6591,1568,7075
Fold_2,8482,5206,2082,8627
Fold_3,7438,5911,1956,6982
Fold_4,9579,6192,2284,6712
Fold_5,8347,6053,2009,7769


In [4]:
match_results = match_table.from_dict(match_dict).T
match_results

,molar,premolar,canine,incisors
Fold_1,5531,4224,1271,5125
Fold_2,5958,3465,1280,6347
Fold_3,5118,4554,1319,5321
Fold_4,5936,4236,1705,4736
Fold_5,5313,4118,1349,5644


## 統計原來各牙齒種類的資料個數

In [5]:
# all_path = f"tmp.csv"
# all_data = pd.read_csv(all_path)

# for tooth_type, tooth_list in tooth_type_dict.items():
#         total_data = all_data[ (all_data["tooth_num"].isin(tooth_list)) ]
#         print( f"Original # of tooth type - {tooth_type} : {len(total_data) // 80 } " )
        
#         for i in range(3):
#                 total_data = all_data[ (all_data["tooth_num"].isin(tooth_list)) & (all_data["Class"] == i) ]
#                 print(f"        # of class {i} : { len(total_data) // 80}")
        

In [6]:
# def accuracy(data, title, filter_list):
#         total_data = data[ (data["tooth_num"].isin(filter_list)) ]
#         match_data = data[ (data["tooth_num"].isin(filter_list)) & (data["Match"] == 1) ]
#         accuracy = round(len(match_data) / len(total_data) * 100, 2)
#         return accuracy